In [4]:
from z3 import *

x = Real('x')
y = Real('y')
s = Solver()
s.add(x + y > 5, x > 1, y > 1)
print(s.check())
print(s.model())

sat
[y = 4, x = 2]


In [5]:
import pandas as pd
pd.set_option('display.max_columns', 500)

housing_data = pd.read_csv('./Data/kc_house_data.csv')

In [6]:
sqft_cols = [i for i in housing_data.columns if 'sqft' in i]

In [7]:
X = housing_data[sqft_cols]
y = housing_data['price']
X.head()

,sqft_living,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15
0,1180,5650,1180,0,1340,5650
1,2570,7242,2170,400,1690,7639
2,770,10000,770,0,2720,8062
3,1960,5000,1050,910,1360,5000
4,1680,8080,1680,0,1800,7503


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn.ensemble import HistGradientBoostingRegressor

gbr = GradientBoostingRegressor(
    max_depth=3,
    learning_rate=0.1,
    n_estimators=50  # number of trees
)

gbr.fit(X,y)


GradientBoostingRegressor(n_estimators=50)

In [9]:
import sklearn
help(sklearn.tree._tree.Tree)

Help on class Tree in module sklearn.tree._tree:

class Tree(builtins.object)
 |  Array-based representation of a binary decision tree.
 |  
 |  The binary tree is represented as a number of parallel arrays. The i-th
 |  element of each array holds information about the node `i`. Node 0 is the
 |  tree's root. You can find a detailed description of all arrays in
 |  `_tree.pxd`. NOTE: Some of the arrays only apply to either leaves or split
 |  nodes, resp. In this case the values of nodes of the other type are
 |  arbitrary!
 |  
 |  Attributes
 |  ----------
 |  node_count : int
 |      The number of nodes (internal nodes + leaves) in the tree.
 |  
 |  capacity : int
 |      The current capacity (i.e., size) of the arrays, which is at least as
 |      great as `node_count`.
 |  
 |  max_depth : int
 |      The depth of the tree, i.e. the maximum depth of its leaves.
 |  
 |  children_left : array of int, shape [node_count]
 |      children_left[i] holds the node id of the left child 

In [10]:
tree = gbr.estimators_[0,0].tree_


In [11]:
tree = gbr.estimators_[0,0].tree_

In [12]:
from copy import deepcopy

In [41]:
# do this just for regression to start

##### HOW TO GENERATE LOGICAL REPRESENTATIONS

### Leaf encoding
# run a DFS on the tree, store nodes in a stack

# when you reach a leaf, write the logical statements for 
# pi(l) which is stack[1:] n wl where wl is the weight value of the leaf

paths = []
pi_l = []  # this will be a list of lists of every pi_l ie: ['xi_1 <= 1000', 'xi_2 > 6000'..., wl]
stack=[0]  # initialize stack for tree
pi_l_stack=[]  # a stack for holding pi_l definitions based on position in tree
n_nodes = tree.node_count
threshold = tree.threshold 
feature = tree.feature
left_searched = [False for _ in range(n_nodes)]
right_searched = [False for _ in range(n_nodes)]
children_left = tree.children_left
children_right = tree.children_right
values = tree.value  # I think these are the average value (or something) of a node based on training data

# DFS
while True:
    this_node = stack[-1]
    this_threshold = threshold[this_node]
    this_feature = feature[this_node]
    
    searched_left_already = left_searched[this_node]
    searched_right_already = right_searched[this_node]
    if searched_left_already and searched_right_already:
        stack.pop(-1)  # pop the last node off of the stack so we go one level up on the next run
        pi_l_stack.pop(-1)
        continue
        
    if not searched_left_already:  # search the left_child
        child = children_left[this_node]
        this_op = f'x_{this_feature}_i <= {this_threshold}'  # I wasn't sure exactly how these should be defined
        left_searched[this_node]=True
    else:  # search the right child
        child = children_right[this_node]
        this_op = f'x_{this_feature}_i > {this_threshold}'
        right_searched[this_node]=True
        
    if child==-1:  # at leaf
        # append path to this node + the value of this node
        this_path = deepcopy(stack[1:])+[deepcopy(values[this_node][0,0])]
        this_pi_l = deepcopy(pi_l_stack[:])+['wl='+deepcopy(str(values[this_node][0,0]))]
        paths.append(this_path)
        pi_l.append(this_pi_l)
        result='And('+', '.join(this_pi_l)+')'
        print(this_path, result)
        
        if this_node == n_nodes-1:
            break  # we have searched all paths and can break
        stack.pop(-1)  # pop the last node off of the stack so we go one level up on the next run
        pi_l_stack.pop(-1)
    else:  # not at leaf
        # append the child to the stack and continue search
        stack.append(child)     
        pi_l_stack.append(this_op)

# pruning: using an input x_i', compare the difference at every node in the DFS x_i
# if |x_i-x_i'| < epsilon where epsilon is the predefined robustness factor
# then keep node, otherwise pop current node from the stack and 
# stop searching this branch of the tree (return to previous level)

### Tree encoding
# then represent PI(D) as V(pi(l)) a disjunction of all pi(l) found during DFS
PI_D = 'Or(\n\t'+',\n\t'.join(['And('+', '.join(this_pi_l)+')' for this_pi_l in pi_l])+'\n)'
print('\nTree Representation:\n',PI_D)
### GBM encoding
# lastly encode the full model instelf, (n(PI(D_i))) n (out=sum(wl_i))
# I haven't fulling figured out how to represent the out=sum(wl_i) part yet.
# wl_i should be the one value returned from each tree PI(D_i), we might just 
# need the additional predicate D_i(x)=wl_i meaning that putting x into the decision tree returns wl_i
# but I'm not totally sure how this is expressed for Z3

### Robust property encoding
# see section 5 of the paper this is easier than the previous stuff but it 
# again has stuff like R(x) that I need to read the documentation on how to define




[1, 2, 3, -191935.98367101545] And(x_0_i <= 3406.0, x_0_i <= 2259.5, x_0_i <= 1529.0, wl=-191935.98367101545)
[1, 2, 4, -84004.33994787265] And(x_0_i <= 3406.0, x_0_i <= 2259.5, x_0_i > 1529.0, wl=-84004.33994787265)
[1, 5, 6, 74673.34801182243] And(x_0_i <= 3406.0, x_0_i > 2259.5, x_4_i <= 2855.0, wl=74673.34801182243)
[1, 5, 7, 281749.63687257515] And(x_0_i <= 3406.0, x_0_i > 2259.5, x_4_i > 2855.0, wl=281749.63687257515)
[8, 9, 10, 452079.9962296856] And(x_0_i > 3406.0, x_0_i <= 4755.0, x_0_i <= 4062.5, wl=452079.9962296856)
[8, 9, 11, 787092.8382778168] And(x_0_i > 3406.0, x_0_i <= 4755.0, x_0_i > 4062.5, wl=787092.8382778168)
[8, 12, 13, 1306390.799409939] And(x_0_i > 3406.0, x_0_i > 4755.0, x_0_i <= 7940.0, wl=1306390.799409939)
[8, 12, 14, 4541341.85823347] And(x_0_i > 3406.0, x_0_i > 4755.0, x_0_i > 7940.0, wl=4541341.85823347)

Tree Representation:
 Or(
	And(x_0_i <= 3406.0, x_0_i <= 2259.5, x_0_i <= 1529.0, wl=-191935.98367101545),
	And(x_0_i <= 3406.0, x_0_i <= 2259.5, x_0_i

In [14]:
tree.value[4][0,0]

-84004.33994787265

In [15]:
tree.children_left


array([ 1,  2,  3, -1, -1,  6, -1, -1,  9, 10, -1, -1, 13, -1, -1],
      dtype=int64)

In [16]:
tree.children_right


array([ 8,  5,  4, -1, -1,  7, -1, -1, 12, 11, -1, -1, 14, -1, -1],
      dtype=int64)

In [17]:
tree